# Bagheri et al. 2013

[Bagheri et al. 2013 - Improving AADT Estimation Accuracy of Short-Term Traffic Counts Using Pattern Matching and Bayesian Statistics](https://ascelibrary.org/doi/10.1061/%28ASCE%29TE.1943-5436.0000528):

MSE Method:
* Strong spatial clustering among traffic annual growth rates of roads, indicating nearby roads have similar annual growth rates.
* Growth rate of closest PTC with same functional class usually has the highest correlation with that of STTC.
* To obtain the seasonal pattern of STTC, we must convert the short-term count collected on a specific day of week in a month to its MADT value.  We do this by determining the ratio between the MADT and average daily traffic for a specific day of week in that month.

$$
\mathrm{DoM}_{dij}=\mathrm{MADT}_{ij}\frac{n_{d,j}}{\sum_{k=1}^{n_{d,j}}\mathrm{DT}_{d,k}}
$$

* $\mathrm{MADT}_{ij}$ is monthly averaged daily traffic in year $i$, month $j$, $\mathrm{DoM}_{dij}$ "day of month factor" for day of week (eg. Monday = 0) $d$, year $i$ and month $j$, $\mathrm{DT}_{d,k}$ is daily traffic for day $d$ of week $k$ of month $j$, $n_{d,j}$ is the number of days of week $d$ in $j$.  For example, for Monday of January 2010, $\mathrm{DoM}$ is the ratio between the MADT of January 2010 with the average daily traffic for all Mondays in January 2010.
* Then, historical STTCs can be converted to MADTs for year of interest $p$ and month $j$ using:

$$
\mathrm{MADT}_{pj} = \mathrm{STTC}_{dij} \times \mathrm{DoM}_{dij} \times \prod_{k=i}^p GR_k
$$

* $\mathrm{STTC}_{dij}$ is the collected short-term traffic count on day of week $d$ of month $j$ and year $i$ (which I assume means it's the average of all days that are Monday, Tuesday, etc.).  $GR_k$ is the annual traffic growth rate for year $k$ obtained from the nearest PTC located on the same functional class
* Bagheri recommends in cases where multiple counts are available for the same month but different years, the best estimate MADT should be an average of those calculated from the above equation.
* Next, we divide $\mathrm{MADT}_{pj}$ by $\mathrm{AADT}_{p}$ to get a normalized seasonal pattern.  Since $\mathrm{AADT}_{p}$ isn't available, we instead estimate the AADT with $\mathrm{AADTprelim}$:

$$
\mathrm{AADTprelim}_p = \frac{\sum \mathrm{STTC}_{dij} \times D_{dij} \times \prod_{k=i}^p GR_k}{n}
$$

* Here, we take the average of all scaled $\mathrm{STTC}_{dij}$ available to us.  The scaling factor $D_{dij}$ comes from the nearest PTC located on the same functional class, and is the annual equivalent to $\mathrm{DoM}_{dij}$:

$$
D_{dij} = AADT_i \frac{n_{d, j}}{\sum_{k=1}^{n_{d, j}}DT_{d,k}}
$$

* The normalized seasonal pattern is then

$$
\mathrm{MF_{pj}} = \frac{\mathrm{MADT}_{pj}}{\mathrm{AADTprelim}_p}
$$

* Assignment of STTC sites using the MSE method is done by computing the value of MSE for all available PTCs:

$$
\mathrm{MSE}^{sm}_p = \frac{1}{N} \sum_j \left(\mathrm{MF^s_{pj}} - \mathrm{MF^m_{pj}}\right)^2
$$

* Here, $s$ is the short term count site, $m$ is the permanent count site, and $N$ the number of data points shared between $s$ and $m$.
* Once the minimum MSE is found, we assign the PTC to the STTC, and find the AADT of the STTC using:

$$
AADT = \mathrm{STTC}_{dij} \times D_{dij}
$$

* Here, we take the most recent STTC observation, and the corresponding day-to-year conversion from the assigned PTC.

## Differences between Bagheri and TEPs-I

* Bagheri uses a growth rate from the nearest PTC, and calculates these rates for year $k$.  In `PTCYEAR.m`, Arman uses the average over all years of the year on year exponential growth rate ($N$ is the number of AADTs we have):

$$
r = \frac{1}{N - 1} \sum_i^{N - 1} \frac{\ln(\mathrm{AADT}_{i+1}/\mathrm{AADT}_i)}{y_{i+1} - y_i}
$$

* Then he calculates a growth rate with $GR = 1 + r$ (which is formally incorrect, but the Taylor expansion works out).
* To calculate $\prod_{k=i}^p GR_k$, he uses $GR^{(p - i)}$.  This smooths out any year-on-year changes in the growth rate.
* Furthermore, in `DoMSTTC.m`, he averages over all PTCs, effectively using the Toronto global PTC growth rate rather than the nearest PTC.
* In `SEL_ratio.m`, he performs an MADT average weighted by the the number of available days in each month that are the same day of week as the STTC day of week.
* Also in `SEL_ratio.m`, he deviates from Eqn. 4 in Bagheri by calculating $D$ as the ratio between AADT and daily count for each day of the year, then averages all $D$ for the same day of week to multiply for $\mathrm{AADTprelim}_p$.
* Because we don't have nearby PTC data for every year we have STTC data, TEPs finds the closest year with same day-of-week (or just closest year if same day-of-week data doesn't exist) data from each of the nearest 5 PTC stations.  The $i$ in Eqn. 2, 3 and 4 in Bagheri is treated as the closest year we have relevant PTC data.  For example, if we're looking at 2006 data that we want to estimate forward to 2018, but the closest PTC year we have is 2016, $\prod_{k=i}^p GR_k = GR^2$, not $GR^{12}$.
* TEPS doesn't consider nearby PTCs only from the same road class (probably because then we'd have too little data).